In [232]:
import time
from itertools import product
from sage.all import matrix, vector, ZZ  # Assuming SageMath for matrix operations
from functools import reduce
import operator
import logging
import time

class Real_Heegaard_Diagram:
    def __init__(self, alphas, betas, tau, alpha_int_beta, alpha_int_C):
        """
        Fix a real Heegaard diagram. Orient and order the alpha curves (which orders and orients the beta curves). 
        Label the intersection points between alphas and betas. We require that intersection points appearing on C are listed first.
        - The inputs alphas and betas are dictionaries whose keys are the indices of the curves and whose values are lists enumerating
        the intersection points appearing along that curve in order according to the orienation of the curve
        - tau is dictionary defined by tau[x] = y, where x is an intersection point and y is its image under the involution
        - Finally, we record the signs of intersection between the alpha and beta curves as well as those between the alpha curves and C.
        This data is enough to partition generators into (real) SpinC structures and to compute the real Euler characteritic. 

        Contained below are useful functions for partitioning generators into SpinC structures and for computing the real Euler characteristic. 
        """
        self.alphas = alphas
        self.betas = betas
        self.tau = tau
        self.alpha_int_beta = alpha_int_beta
        self.alpha_int_C = alpha_int_C
        
        # Compute unordered_alphas and unordered_betas by sorting the values
        self.unordered_alphas = {k: sorted(v) for k, v in alphas.items()}
        self.unordered_betas = {k: sorted(v) for k, v in betas.items()}

        # Compute number_pts_on_C based on tau (i.e., count the fixed points)
        self.fixed_points = [k for k, v in self.tau.items() if k == v]
        self.number_pts_on_C = len(self.fixed_points)

        # Dictionary associating the label of an alpha curve to its index
        self.handle_to_index = {key: index for index, key in enumerate(alphas.keys())}

        # Dictionary assigning an intersection point the labels of the alpha and beta curve it lies on
        self.intersection_to_curve_pair = {}

        # Populate the dictionary with elements and their corresponding alphas key
        for a_key, a_list in self.alphas.items():
            for element in a_list:
                if element not in self.intersection_to_curve_pair:
                    self.intersection_to_curve_pair[element] = [None, None]  # Initialize with placeholders
                self.intersection_to_curve_pair[element][0] = a_key  # Set the alpha key

        # Populate the dictionary with elements and their corresponding betas key
        for b_key, b_list in self.betas.items():
            for element in b_list:
                if element not in self.intersection_to_curve_pair:
                    self.intersection_to_curve_pair[element] = [None, None]  # Initialize with placeholders
                self.intersection_to_curve_pair[element][1] = b_key  # Set the beta key


    def beta_combos(self):
        """
        The first homology of Y is determined by the intersections between alphas and betas. This builds that matrix
        """
        combos = []
        for beta_curve in self.betas:
            v = []
            for alpha_curve in self.alphas:
                pts = [
                    x for x in self.betas[beta_curve] 
                    if x in self.alphas[alpha_curve]
                ]
                lst = [self.alpha_int_beta[x] for x in pts]
                v.append(sum(lst))      
            combos.append(v)
        return combos

    def homology(self): 
        """
        returns the first homology (as the invariant factors)
        """
        beta_comb = self.beta_combos()
        n = len(beta_comb[0])
        B_matrix = matrix(ZZ, beta_comb).transpose()

        # Find Smith normal form
        S, U, V = B_matrix.smith_form()

        # Extract non-trivial invariant factors
        invariants = [S[i, i] for i in range(min(S.nrows(), S.ncols())) if S[i, i] != 0]
        orders = [x for x in invariants if x != 1]

        if len(orders) == 0:
            return [1]
        return orders

    def loop(self, x, y, return_paths=False):
        """
        Given a generator, finds loop between them (the class ep(x,y) which is dual to the difference of the spinC structures given by x and y)
        """
        alpha_curve_list = list(self.unordered_alphas.keys())
        g = len(alpha_curve_list)
        point_to_curves = self.intersection_to_curve_pair
        
        pathA = [] 
        pathB = []
        
        for x0 in x:
            al = self.alphas[point_to_curves[x0][0]]
            be = self.betas[point_to_curves[x0][1]]
            
            y0 = list(set(al).intersection(y))[0]
            z0 = list(set(be).intersection(y))[0]
            n = len(al)
            
            ind_x = al.index(x0)
            ind_y = al.index(y0)

            if return_paths:
                if abs(ind_x - ind_y) < n/2:
                    if ind_x < ind_y:
                        path = ["+"] + al[ind_x:ind_y + 1]
                    else:
                        path = al[ind_y:ind_x + 1]
                        path.reverse()
                        path = ["-"] + path
                else:
                    if ind_x < ind_y:
                        temp1 = al[:ind_x+1]
                        temp2 = al[ind_y:]
                        temp1.reverse()
                        temp2.reverse()
                        path = ["-"] + temp1 + temp2
                    else:
                        path = ["+"] + al[ind_x:] + al[:ind_y+1]
                pathA.append(path)
            
            ind_y = be.index(z0)
            ind_x = be.index(x0)
            
            if abs(ind_y - ind_x) < n/2:
                if ind_y < ind_x:
                    path = ["+"] + be[ind_y:ind_x + 1]
                else:
                    path = be[ind_x:ind_y + 1]
                    path.reverse()
                    path = ["-"] + path
            else:
                if ind_y < ind_x:
                    temp1 = be[:ind_y+1]
                    temp2 = be[ind_x:]
                    temp1.reverse()
                    temp2.reverse()
                    path = ["-"] + temp1 + temp2
                else:
                    path = ["+"] + be[ind_y:] + be[:ind_x+1]
            pathB.append(path)

        combos = [0] * g
        
        for trajectory in pathB:
            if len(trajectory) != 2:
                orientation = 1 if trajectory[0] == "+" else -1
                
                path = trajectory[1:] 
                start = path[0]
                end = path[-1]
                
                start_alpha = point_to_curves[start][0]
                start_index = alpha_curve_list.index(start_alpha)
                
                end_alpha = point_to_curves[end][0]
                end_index = alpha_curve_list.index(end_alpha)
        
                combos[start_index] += 0.5 * orientation * self.alpha_int_beta[start]
                combos[end_index] += 0.5 * orientation * self.alpha_int_beta[end]
        
                for intersection in path[1:-1]:
                    alpha_curve = point_to_curves[intersection][0]
                    alpha_index = alpha_curve_list.index(alpha_curve)
                    combos[alpha_index] += orientation * self.alpha_int_beta[intersection]

        if return_paths:
            return pathA, pathB, combos
        else:
            return combos

    def gen_to_spinC(self, x, y):
        '''
        Given generators x and y, returns their difference as an element of H_1 (as well as |H_1|)
        This only makes sense if x is fixed for all y.
        '''
        # Find the span of the beta curves in H_1.
        beta_comb = self.beta_combos()
        n = len(beta_comb[0])
        B_matrix = matrix(ZZ, beta_comb).transpose()

        # Find Smith normal form
        S, U, V = B_matrix.smith_form()

        # Extract non-trivial invariant factors
        invariants = [S[i, i] for i in range(min(S.nrows(), S.ncols())) if S[i, i] != 0]

        # Find ep(x, y) (i.e. a loop between them)
        xy_loop = self.loop(x, y)

        xy_vector = vector(ZZ, xy_loop)
        
        # Transform the element into canonical form using the matrix U
        canonical_form = U * xy_vector
        
        # Reduce modulo invariants and form the residue class
        residue_class = tuple(
            (canonical_form[i] % invariants[i]) if i < len(invariants) else canonical_form[i] 
            for i in range(n)
        )
        # The quotient is a direct sum of Z/m_i, where m_i | m_{i+1}. 
        orders = [x for x in invariants if x != 1]

        if len(orders) == 0:
            return (0,), [1]
        else:
            # residue_class = (0, 0, ..., m_1,..., m_k). We just want the last k elements.
            reduced_residue_class = residue_class[-len(orders):]
    
            #print(orders, reduced_residue_class)
            return reduced_residue_class, orders
            
    def epsilon2(self, g):
        """
        This computes the quantity epsilon, which is the sign of the permutation given by comparing the 
        chosen ordering of the alpha curves to the ordering given by a generator
        """
        N = self.number_pts_on_C

        pt_to_curves = self.intersection_to_curve_pair
        
        r = {}
        s = {}
        sigma = {}

        C = [x for x in g if x in self.fixed_points]
        Z = [x for x in g if x not in C]        
        
        C.sort()
        Z.sort()
        
        index_handle = self.handle_to_index
        #print(Z,C)

        for i in range(len(Z)//2):
            alpha_index = index_handle[pt_to_curves[Z[2*i]][0]]
            r[i] = alpha_index
            
        for i in range(len(Z)):
            alpha_index = index_handle[pt_to_curves[Z[i]][0]]
            beta_index = index_handle[pt_to_curves[Z[i]][1]]
            
            sigma[alpha_index] = beta_index
                  
        
        for i in range(len(C)):
            alpha_index = index_handle[pt_to_curves[C[i]][0]]
            s[i] = alpha_index
        #print(s)
        
        ep = []
        
        if len(r) > 0:
            for i in range(len(Z)//2):
                ep.append(sigma[r[i]])
                ep.append(r[i])
        if len(C) > 0:
            for i in range(len(C)):
                ep.append(s[i])
        
        #print("perm:",ep)
        #print("ep:",permutation_sign(ep))
        if len(set(ep)) < len(ep):
            print("Permutation error!")
            print("generator=",g)
            print("tuple=",ep)
        
        return permutation_sign(ep), C, Z
    
    def intersection_sign(self, x):
        """
        Computes the sign of the intersection between T_a and M^R at x. 
        """
        N = self.number_pts_on_C
        a_int_c = self.alpha_int_C

        g = list(x)
        g.sort()
        #print(g)
        
        #print("ep:")
        ep, C, Z = self.epsilon2(g)
        
        k = len(Z) // 2  # Ensure integer division if Python 2
        #print("k:",k)
        prod = 1
        for i in g:
            #print("pt:",i)
            if i in C:
                prod *= a_int_c[i]
                #print("(i, a*c(i)):", i, a_int_c[i])
            else:
                if Z.index(i) % 2 == 0:
                    #print("(i, a*c(i)):", i, a_int_c[i])
                    prod *= a_int_c[i]
        return ((-1)**k) * ep * prod
        #return ep * prod
    

    def transform_set(self, S):
        """
        Given a set S, returns tau(S).
        """
        return {self.tau[s] for s in S if s in self.tau}

    def generate_set_of_sets_with_tau(self, list_of_lists, milestones = False):
        """
        Given a list of lists (alpha curves, say) returns tau-invariant unordered g-tuples 
        Also, optionally prints milestones.
        """
        prepared_lists = [list(lst) if isinstance(lst, (list, tuple, set)) else [lst] for lst in list_of_lists]

        # Milestones
        element_counter = 0
        milestone = 1_000_000_000
        
        for combination in product(*prepared_lists):
            fs_combination = frozenset(combination)

            #Increment element counter
            element_counter += 1
            if element_counter % milestone == 0:
                if milestones == True:
                    print(f"Processed {element_counter} elements for beta")
            
            if fs_combination == self.transform_set(fs_combination):
                yield fs_combination


    def real_euler_characteristics(self, return_runtime=False, milestones = False):
        """
        Computes the real Euler characteristic. 
        """
        start_time = time.time()  # Record the start time
        if milestones == True:
            print("Starting computation...")
        
        # Collect all tau-invariant elements from S_A ∩ S_B
        C = self.gen_to_spinC
        beta_curves = [self.betas[x] for x in self.betas]
        tau_invariant_S_B = set(self.generate_set_of_sets_with_tau(beta_curves))
        tau_invariant_intersection = set()
        
        if milestones == True:
            print("Invariant beta points obtained...")
        
        # Milestones
        alpha_keys = list(self.alphas.keys())
        element_counter = 0
        milestone = 1_000_000_000  # Example milestone count
    
        def backtrack(index, current_tuple):
            nonlocal element_counter
            if index == len(alpha_keys):
                frozen_tuple = frozenset(current_tuple)
                if frozen_tuple in tau_invariant_S_B and frozen_tuple == self.transform_set(frozen_tuple):
                    tau_invariant_intersection.add(frozen_tuple)
                return
            alpha_key = alpha_keys[index]
            for element in self.alphas[alpha_key]:
                current_tuple.append(element)
                backtrack(index + 1, current_tuple)
                current_tuple.pop()

                # Increment element counter
                element_counter += 1
                if element_counter % milestone == 0:
                    if milestones == True:
                        print(f"Processed {element_counter} elements for alpha points.")
    
        # Find all tau-invariant elements first
        backtrack(0, [])
        logging.info("All tau-invariant intersections found.")

        # Pick an x_0 from tau_invariant_intersection
        x_0 = next(iter(tau_invariant_intersection))
    
        # Initialize class_sums based on the number of equivalence classes
        _, orders = C(x_0, x_0)  # Assuming C(x_0, x_0) gives the second output as the list of orders
        classes = list(product(*(range(m) for m in orders)))
        class_sums = {x: 0 for x in classes}

        euler_start_time = time.time()
        if milestones == True:
            print(f"Time to prepare for Euler characteristic computation: {euler_start_time - start_time:.2f} seconds.")
            print("Computing Euler characteristics...")
        
        # Process each element and categorize using C
        element_counter = 0  # Reset counter for Euler computation
        for y in tau_invariant_intersection:
            class_index, _ = C(x_0, y)
            class_sums[class_index] += self.intersection_sign(y)

            # Increment element counter
            element_counter += 1
            if element_counter % milestone == 0:
                if milestones == True:
                    print(f"Processed {element_counter} elements for Euler computation.")

        # Measure the end time and calculate runtime
        end_time = time.time()
        runtime = end_time - start_time
        if milestones == True:
            print(f"Total runtime: {runtime:.2f} seconds.")
        # Finally, if H_1 is cyclic, we reorder the SpinC structures so the symmetric one is in the middle
        
        if len(orders)  == 1:
            spinCs = list(class_sums.values())
            while not is_symmetric(spinCs):
                spinCs = shift_list(spinCs)
            if sum(spinCs) > 0:
                return spinCs
            else:
                return [-1*x for x in spinCs]
        else:
            print('no')
        if return_runtime:
            return class_sums, runtime
        else:
            return class_sums

    def real_total_euler_characteristics(self, return_runtime=False, milestones=False):
        start_time = time.time()  # Record the start time
        if milestones == True:
            print("Starting computation...")
        
        # Collect all tau-invariant elements from S_A ∩ S_B
        C = self.gen_to_spinC
        beta_curves = [self.betas[x] for x in self.betas]
        tau_invariant_S_B = set(self.generate_set_of_sets_with_tau(beta_curves))
        tau_invariant_intersection = set()
        
        if milestones == True:
            print("Invariant beta points obtained...")
        
        # Milestones
        alpha_keys = list(self.alphas.keys())
        element_counter = 0
        milestone = 1_000_000_000  # Example milestone count
    
        def backtrack(index, current_tuple):
            nonlocal element_counter
            if index == len(alpha_keys):
                frozen_tuple = frozenset(current_tuple)
                if frozen_tuple in tau_invariant_S_B and frozen_tuple == self.transform_set(frozen_tuple):
                    tau_invariant_intersection.add(frozen_tuple)
                return
            alpha_key = alpha_keys[index]
            for element in self.alphas[alpha_key]:
                current_tuple.append(element)
                backtrack(index + 1, current_tuple)
                current_tuple.pop()

                # Increment element counter
                element_counter += 1
                if element_counter % milestone == 0:
                    if milestones == True:
                        print(f"Processed {element_counter} elements for alpha points.")
    
        # Find all tau-invariant elements first
        backtrack(0, [])
        logging.info("All tau-invariant intersections found.")


        euler_start_time = time.time()
        if milestones == True:
            print(f"Time to prepare for Euler characteristic computation: {euler_start_time - start_time:.2f} seconds.")
            print("Computing Euler characteristics...")
        
        # Process each element and categorize using C
        element_counter = 0  # Reset counter for Euler computation
        total_euler = 0
        for y in tau_invariant_intersection:
            total_euler += self.intersection_sign(y)

            # Increment element counter
            element_counter += 1
            if element_counter % milestone == 0:
                if milestones == True:
                    print(f"Processed {element_counter} elements for Euler computation.")

        # Measure the end time and calculate runtime
        end_time = time.time()
        runtime = end_time - start_time
        
        if milestones == True:
            print(f"Total runtime: {runtime:.2f} seconds.")
        if return_runtime:
            return total_euler, runtime
        else:
            return abs(total_euler)

### Helper functions

def permutation_sign(perm):
    inversions = 0
    n = len(perm)
    
    # Count the number of inversions
    for i in range(n):
        for j in range(i + 1, n):
            if perm[i] > perm[j]:
                inversions += 1
    
    # Determine the sign based on the number of inversions
    return 1 if inversions % 2 == 0 else -1

def shift_list(list):
    """
    shifts all entries of list to the right
    """
    new_list = []
    n = len(list)
    for i in range(n):
        new_list.append(list[(i+1)%n])
    return new_list

def is_symmetric(list): 
    """
    given a list, returns true if list[-i] = list[i]
    """
    n = len(list)
    for i in range(n//2):
        if list[i] != list[n-i-1]:
            return False
    return True
    

In [233]:
"""
Below is some code which will generate the data needed to specify a real Heegaard diagram 
for the branched double cover of a knot in S^3 by representing K as a braid closure.
While such Heegaard diagrams are not always the most efficient (the genus is sometimes large), the intersection between the 
resulting alphas and betas are determined by the braid word.
"""


def br(x, s, n): 
    """
    Given a generator of the braid group x and a position on the braid (i.e. a pair, [i, j] where i is a strand and j is the crossing), returns the state obtained by crossing x. 
    Also returns whether the state traveled under or over the crossing and whether the strand index increased or decreased. n is the length of the braid word
    """
   
    i_new = 0
    j_new = (s[1]+1)%n
    if s[0] == abs(x)-1:
        i_new = s[0]+1
        if x > 0:
            return [i_new,j_new], +1, +1 # returns the new state, whether it was an under (-1) or over (+1) crossing, and whether i increased or decreased by 1.
        else:
            return [i_new,j_new], -1, +1
            
    elif s[0] == abs(x):
        i_new = s[0]-1
        if x > 0:
            return [i_new,j_new], -1, -1
        else:
            return [i_new,j_new], +1, -1
    else:
        return [s[0], (s[1]+1)%n], 0, 0

def cross(w, x, s): 
    """
    given a generator x and strand s, returns x(s). Keeps track of less than br function. 
    """
    if abs(x) == s:
        return s-1
    elif abs(x) == s+1:
        return s+1
    else:
        return s

def sign(x):
    """
    Returns the sign of the crossing of the x entry in a braid word
    """
    if x == 0:
        return "Number must be nonzero"
    if x > 0:
        return 1
    else:
        return -1
        
def num_gens(w): 
    """
    given a braid word, returns the number of generators
    """
    v = set([abs(x) for x in w])
    return len(v)

def count_handles(w, i): 
    """
    given a braid word and generator i of braid group returns the number of times i or -i appears. 
    """
    count = 0
    for j in w:
        if abs(j) == abs(i):
            count+= 1
    return count


def distinguished_handles(w): 
    """
    given a word, returns the indices of the distinguished handles
    """
    n = num_gens(w)
    l = []
    for i in range(1,n+1):
        l.append(next(j for j in range(len(w)) if abs(w[j]) == i))
    return l

def distinguished_pair(w, i): 
    """
    given an handle index i which is not distinguished, returns the index of the distinguished handle
    """
    for j in range(len(w)):
        if abs(w[j]) == abs(w[i]):
            return j
    

def list_alpha_curves(w): 
    """
    given a word w, returns a list of alphas circles which are index by the non-distinguished handles
    """
    al = [i for i in range(len(w))]
    for x in distinguished_handles(w):
        al.remove(x)
    return al

def handle_to_index(w): 
    """
    the alpha curves are indexed by the nondistinguished handle they pass through. Returs a dictionary relating the handle index to the set of alpha curves passing through
    """
    index = {}
    L = list_alpha_curves(w)
    for i in range(len(L)):
        index[L[i]] = i
    return index

## Given a braid word, there is a canonical free Seifert surface. Each letter becomes a handle. The first instance of generator i is a distinguished handle, 
## and we pick alpha curves to pass through this handle and each other handle corresponding to a handle labeled i

def int_regions(w): 
    """
    given a word w, outputs a dictionary whose keys are handles and arcs between handles in the diagram. 
    The values are dictionaries whose keys are the alpha curves in that region, and values are empty lists, to be populated later. 
    """
    d = {}
    for i in range(num_gens(w)): # i runs over the seifert disks (or equivalently, strands in the braid word
        ## First, we add in the handles which from from disk i to i+1. These correspond to generators (i+1) in w.
        for x in handle_indices(w, i+1):
            if x in distinguished_handles(w):
                A = (handle_indices(w,w[x]))[1::]
                d[x] = {p:[] for p in A}
            else:
                d[x]= {x:[]}
                
        # Next, we look at curves which pass the fixed set between handles i and j on strand s
    for s in range(1, num_gens(w)):
        arcs = enum_disk_arcs(w,s)[0]
        alphas = label_disk_ints(w,s)
        for i in range(len(arcs)):
            A = tuple(arcs[i])
            d[A] = {x: [] for x in alphas[i]}
    return d

def combine_dicts(dict_list):
    """
    Given a list of dictionaries, combines them into a single one
    """
    combined_dict = {}
    for d in dict_list:
        combined_dict.update(d)  # Update the combined dictionary with each dictionary
    return combined_dict


def filter_dict_values(D, A):
    """
    Removes all elements of list A from the values of dictionary D.
    """
    return {key: [item for item in value if item not in A] for key, value in D.items()}

def remove_keys_from_dict(D, A):
    """
    Removes keys from the dictionary D that are present in the list A.
    """
    return {key: value for key, value in D.items() if key not in A}

def knot_path(w, test = False): #
    """
    given a word w whose braid closure is a knot K, returns the order in which crossings are passed while traversing the knot. Also records whether the traversal was over or under the crossing
    as a simple test, can check that the path length is twice the length of the braid word and that the signs add up to zero.
    """
    s = [0,0]
    P = []
    n= len(w)
    m = num_gens(w)+1
    for i in range(n*m):
        out = br(w[i%n], s, n)
        if out[1] != 0:
            P.append([i%n,out[1]])
        s = br(w[i%n], s, n)[0]
    if test == True:
        return len(P) == n*2 and sum([x[1] for x in P]) == 0
    return P

def link_path(w, test = False): 
    """
    given a word w whose braid closure is a link L, returns the order in which crossings are passed while traversing the knot. Also records whether the traversal was over or under the crossing
    as a simple test, can check that the path length is twice the length of the braid word and that the signs add up to zero.
    """
    initial_state = [0,0]
    path = []
    n= len(w)
    m = num_gens(w)+1
    traversed_strands = set() # this is a list of the strands that have been traversed. When this list contains all strands, the proceedure stop. 
    
    while len(traversed_strands) < m:
        current_strand = min([strand for strand in range(m) if strand not in traversed_strands])
        initial_state = [current_strand,0]
        traversed_strands.add(current_strand)
        #print("current_strand=", current_strand)
        
        current_crossing = 0
        initial_state = [current_strand, current_crossing] 
        current_state, current_crossing_sign, current_veering =  br(w[current_crossing%n], initial_state, n)
        #print('inital state=', initial_state)

        if current_crossing_sign!= 0:
            path.append([current_crossing%n, current_crossing_sign])
        
        while current_state != initial_state:
            current_crossing+=1
            current_state, current_crossing_sign, current_veering =  br(w[current_crossing%n], current_state, n)
            #print('current_state=', current_state)
            
            if current_crossing_sign!= 0:
                path.append([current_crossing%n, current_crossing_sign])
            if current_state[1] == 0:
                traversed_strands.add(current_state[0])
                #print(traversed_strands)
                  
    return path


def num_handle_arcs(w,i, signs = False): 
    """
    given a braid word w and integer i < len(w), returns the number of alpha-arcs running through the ith handle of the corresponding Heegaard splitting 
    """
    if i in distinguished_handles(w):
        return count_handles(w,w[i]) - 1
    else:
        if signs == True:
            return -1
        else:
            return 1

def handle_indices(w, x): 
    """
    given a braid word and generator x>0, returns the indices of letters in w which are equal to +/-x
    """
    ind = []
    for i in range(len(w)):
        #print(i,w[i])
        if abs(x) == abs(w[i]):
            ind.append(i)
    return ind

def enum_disk_arcs(w, i, handlelist = False): 
    """
    Given w, returns a list of the number of alpha-arcs which cross the knot between each pair of handles
    On the ith disk, there are handles attached for each i generator as well as for each i+1 generator. 
    """
    if i > num_gens(w):
        return 'i is greater than the number of disks'
    n = len(w)
    handles = []
    for j in range(n):
        if abs(w[j]) in [i, i+1]:
            handles.append(j)
    dist = distinguished_handles(w)
    pts = []
    sum = 0
    if i ==0 or i == (num_gens(w)):
        if handlelist == True:
            return handles
        else:
            return [[[handles[i], handles[i+1]] for i in range(len(handles)-1)], [0 for i in range(1,len(handles))]]
    for j in range(1,len(handles)):
        h = handles[j]
        sum+= num_handle_arcs(w,handles[j-1],True)
        if abs(w[h]) == abs(w[h-1]):
            pts.append(0)
        else:
            pts.append(sum)
    if handlelist == True:
        return handles
    else:
        return [[handles[i], handles[i+1]] for i in range(len(handles)-1)],pts



def label_disk_ints(w,i): 
    """
    a function that returns a list of lables of intersection points appearing in each arc on the ith disk
    """
    handles = enum_disk_arcs(w,i,True)
    arcs = enum_disk_arcs(w,i)[0]
    labels = []
    gens = [abs(w[x]) for x in handles]
    dist = distinguished_handles(w)
    m=min(gens)
    l = []
    same = True
    if i == 0 or i == num_gens(w):
        return [[] for i in arcs]
    for j in range(len(handles)-1):
        #print('j=',j)
        #print(labels)
        h = handles[j]
        if h in dist: #in this case, we add strands to our list
            I = handle_indices(w,w[h])[1::]
            if abs(w[h]) == m:
                l = I+l
                #print(l)
                if abs(w[h]) != abs(w[handles[j+1]]): 
                    labels.append(l.copy())
                    l.reverse() # when the generators change, there is a finger move that reversed the order of all generators
            else:
                l = I+l
                #print(l)
                if abs(w[h]) != abs(w[handles[j+1]]): 
                    labels.append(l.copy())
                    l.reverse()
            
            if abs(w[h]) == abs(w[handles[j+1]]): 
                labels.append([])
        else: # in this case, strands are being removed
            if abs(w[h]) == abs(w[handles[j+1]]): 
                l.remove(h)
                #print(l)
                labels.append([])
            else:            
                l.remove(h)
                #print(l)
                labels.append(l.copy())
                l.reverse()
    return labels


## Santity checks 

def count_int_pts(w): 
    """
    as a santity check counts how many intersection points there should be
    """
    count = 0
    for i in range(len(w)):
        count += num_handle_arcs(w,i)**2
    for i in range(num_gens(w)+1):
        for a in label_disk_ints(w,i):
            count += len(a)**2
    return count

def count_fix_pts(w): 
    """
    sanity check -- counts the number of intersection points appearing on the knot
    """
    count = 0
    for i in range(len(w)):
        count += num_handle_arcs(w,i)
    for i in range(num_gens(w)+1):
        for a in label_disk_ints(w,i):
            count += len(a)
    return count

def count_alphas(w): 
    """
    Returns the number of alpha curves
    """
    count = 0
    n = num_gens(w)
    for i in range(n):
        count+= count_handles(w,i)
    return count


"""
Here is the main function. Given a braid word whose closure is K, make_HD returns the data needed to define a real Heegaard diagram as above. 
"""

def make_HD(w, simplify = True, test = False): 
    """
    returns a list of alpha and beta circles, as well as a dictionary defining the involution.
    If `simplify' is set to True, the function will try to remove cancelling intersection points.
    If `test' is set to true, will run the above santity check functions.
    """
    g = len(list_alpha_curves(w))
    A = {}
    al = {i: [] for i in list_alpha_curves(w)}
    be = {i: [] for i in list_alpha_curves(w)}
    tau={}
    ab={} # signs of intersections between alphas and beta
    ac={} # signs between alphas and the fixed set C
    seifert = {} # tells whether a point is in S^+, S^-, or C = S^+ \cap S^-
    fixpt = 0
    dist = distinguished_handles(w)
    regions = int_regions(w) # this is a dictionary whose keys are handles i or regions between handles (i, i+1). This function will output the alpha curves through that region and the points on them
    regionsB = int_regions(w) # same but for the betas
    
    # First, we list all generators which lie on the fixed set, following the orientation of the knot, and add them to the appropriate alpha/beta circle
    P = link_path(w)
    n= len(w)
    m = num_gens(w)+1 # number of seifert disks
    disk_arcs = [enum_disk_arcs(w,i)[0] for i in range(num_gens(w)+1)] ## disk_arcs[i] is a list of regions between handles.
    s = 0
    
    for i in range(len(P)):
        #print('path index = ',P[i][0])
        #print('strand = ',0)
        x0 = (P[i%(2*n)])[0]
        x1 = (P[(i+1)%(2*n)])[0]
        if P[i][1] == 1: # intersection points only appear at over-crossings
            #first, we add intersection points arising from crossing x0
            
            if P[i][0] in dist:
                alphas = (handle_indices(w,w[x0]))[1::] # there is an alpha curve for every handle with same braid generator as x0
                alphas.reverse()
                for a in alphas:
                    (al[a]).append(fixpt)
                    (be[a]).append(fixpt)
                    ((regions[P[i][0]])[a]).append(fixpt)
                    ((regionsB[P[i][0]])[a]).append(fixpt)

                    ac[fixpt]=1
                    ab[fixpt] = sign(w[x0])
                    A[fixpt] = a
                    seifert[fixpt]=0
                    fixpt+=1
                    
            else:
                (al[P[i][0]]).append(fixpt)
                (be[P[i][0]]).append(fixpt)
                ((regions[P[i][0]])[P[i][0]]).append(fixpt)
                ((regionsB[P[i][0]])[P[i][0]]).append(fixpt)
                ac[fixpt]=-1
                ab[fixpt] = sign(w[P[i][0]])
                A[fixpt] = P[i][0]
                seifert[fixpt]=0
                fixpt+=1
        #print(A)
        
    # Having added intersection points from crossings, we now add intersection points appearing in the disk arc between x0 and x1
        
        s = cross(w, w[x0],s) # here, s is the strand, or equivalently, the Seifert disk
        #print(s)
        if x0 < x1:
            arc = [x0,x1]
            D = disk_arcs[s]
            alphas = label_disk_ints(w,s)
            #print(alphas)
            #print('arc=',arc)
            #print('D=',D)
            #print('alphas=',alphas)
            k = D.index(arc)
            #print('k=',k)
            if len(alphas)>0:
                for a in alphas[k]: # this is a list of the alpha curves which pass through this arc (in order)
                    if abs(w[x1])>abs(w[x0]):
                        if abs(w[a]) == s:
                            ac[fixpt] = -1
                            ab[fixpt] = -1
                        else:
                            ac[fixpt] = 1
                            ab[fixpt] = -1
                    else:
                        if abs(w[a]) == s:
                            ac[fixpt] = 1
                            ab[fixpt] = 1
                        else:
                            ac[fixpt] = -1
                            ab[fixpt] = 1
                    (al[a]).append(fixpt)
                    (be[a]).append(fixpt)
                    ((regions[(x0,x1)])[a]).append(fixpt)
                    ((regionsB[(x0,x1)])[a]).append(fixpt)
                    A[fixpt] = a
                    seifert[fixpt]=0
                    fixpt+=1


    # All intersection points on the fixed set have been labeled. Moreover, x is in A[i] iff x is in B[i] for the fixed set. Obviously tau = id for these
    B = A.copy()
    for i in range(fixpt):
        tau[i] = i
    N = fixpt

    # Now we again traverse the knot, but now add intersection points which are not in the fixed set. We add this points in pairs, always listing the point in \Sigma^+ first
    #print('done with fixed set')
    s = 0
    for i in range(len(P)):
        x0 = (P[i%(2*n)])[0]
        x1 = (P[(i+1)%(2*n)])[0]
        if P[i][1] == 1: # intersection points only appear at over-crossings
            #first, we add intersection points arising from crossing x0
            #print(fixpt, 'handle')
            #print('path index = ',P[i][0])
            if P[i][0] in dist:
                alphas = (handle_indices(w,w[x0]))[1::] # there is an alpha curve for every handle with same braid generator as x0
                alphas.reverse()
                for j in range(len(alphas)-1): # here, we add intersection points in pairs (which are exchanged by tau). If n alpha curves pass the handles, there are n^2 intersection points (though n are on the knot so have already been added.)
                    for k in range(1, len(alphas)-j):
                        tau[fixpt] = fixpt +1
                        tau[fixpt +1] = fixpt
                        if w[x0]>0:  ## Here, we do the case of a positive crossing. 
                            ac[fixpt]=1
                            ab[fixpt]=1
                            ac[fixpt+1]=1
                            ab[fixpt+1]=1
    
                            (al[alphas[j+k]]).append(fixpt)
                            (be[alphas[j]]).append(fixpt)

                            ((regions[P[i][0]])[alphas[j+k]]).append(copy(fixpt))
                            ((regionsB[P[i][0]])[alphas[j]]).append(copy(fixpt))
                            
                            
                            A[fixpt] = alphas[j+k]
                            B[fixpt] = alphas[j]
                            seifert[fixpt]=1
                            fixpt+= 1

                            (al[alphas[j]]).append(copy(fixpt))
                            (be[alphas[j+k]]).append(copy(fixpt))
                            
                            ((regions[P[i][0]])[alphas[j]]).append(copy(fixpt))
                            ((regionsB[P[i][0]])[alphas[j+k]]).append(copy(fixpt))
                            
                            
                            A[fixpt] = alphas[j]
                            B[fixpt] = alphas[j+k]
                            seifert[fixpt]=-1
                            fixpt+= 1
                            
                        else: ## Negative crossings
                            ac[fixpt]=-1
                            ac[fixpt+1]=-1
                            ab[fixpt]=-1
                            ab[fixpt+1]=-1
                            
                            (al[alphas[j]]).append(copy(fixpt))
                            (be[alphas[j+k]]).append(copy(fixpt))
                            
                            ((regions[P[i][0]])[alphas[j]]).append(copy(fixpt))
                            ((regionsB[P[i][0]])[alphas[j+k]]).append(copy(fixpt))
                            ## betas
                            
                            
                            A[fixpt] = alphas[j]
                            B[fixpt] = alphas[j+k]
                            seifert[fixpt]=1
                            fixpt+= 1
    
                            (al[alphas[j+k]]).append(fixpt)
                            (be[alphas[j]]).append(fixpt)

                            ((regions[P[i][0]])[alphas[j+k]]).append(copy(fixpt))
                            ((regionsB[P[i][0]])[alphas[j]]).append(copy(fixpt))
                            
                            A[fixpt] = alphas[j+k]
                            B[fixpt] = alphas[j]
                            seifert[fixpt]=-1
                            fixpt+= 1
                        
                        
        # no 'else' here, since non-distinguished handles only have one intersection point which has already been added.
        
        # Now we do disk arcs. Same idea here -- if n alpha arcs cross the knot along this arc, there will be n^2 intersection points, n of these already have been added. 
        s = cross(w, w[x0],s) # here, s is the strand, or equivalently, the Seifert disk
        arc = [x0,x1]
        if x0 < x1:
            #print(fixpt,'arc')
            D = disk_arcs[s]
            k = D.index(arc)
            alphas = label_disk_ints(w,s)[k]
            #print(alphas)
            
            for j in range(len(alphas)-1):
                for l in range(1,len(alphas)-j):
                    ## Each of these regions contain n^2 - n intersection points, and we add them in symmetric pairs. 
                    ## l ranges over the beta curves that the alphas intersect
                    tau[fixpt] = fixpt +1
                    tau[fixpt+1] = fixpt

                    ## There are two cases to consider, depending on whether the alpha curves curve up or down by handle x_1. 
                    ## This is determined by the relative values of w[x_0] and w[x_1]: if w[x_1] is larger, the alphas curve up above the handle, and otherwise the curve below the handle. 
                    ## This determines how we label the 
                    
                    if abs(w[x1]) > abs(w[x0]): 
                        ep = 1
                        if abs(w[alphas[j]]) == abs(w[alphas[j+l]]):
                            ## Every alpha (and beta) curve in these regions pass through the handles which correspond to the braid group generator [i-1] or [i]
                            ## The sign of the intersection depends on whether the intersection belongs to the same generator or one of each.
                            ac[fixpt]= -ep
                            ac[fixpt+1]= -ep
                            ab[fixpt]= -ep
                            ab[fixpt+1]= -ep
                            (al[alphas[j]]).append(copy(fixpt))
                            (be[alphas[j+l]]).append(copy(fixpt))
    
                            ((regions[(x0,x1)])[alphas[j]]).append(copy(fixpt))
                            ((regionsB[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            
                            
                            A[fixpt] = alphas[j]
                            B[fixpt] = alphas[j+l]
                            seifert[fixpt]=1
                        
                            fixpt+= 1
                        
                            A[fixpt] = alphas[j+l]
                            B[fixpt] = alphas[j]
                            seifert[fixpt]=-1
        
                            (al[alphas[j+l]]).append(copy(fixpt))
                            (be[alphas[j]]).append(copy(fixpt))
    
                            ((regions[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            ((regionsB[(x0,x1)])[alphas[j]]).append(copy(fixpt))
        
                            
                            fixpt+= 1
                            #print(alphas[j])
                        else:
                            ac[fixpt]= ep
                            ac[fixpt+1]= ep
                            ab[fixpt]= ep
                            ab[fixpt+1]= ep
                            
                            (al[alphas[j]]).append(copy(fixpt))
                            (be[alphas[j+l]]).append(copy(fixpt))
    
                            ((regions[(x0,x1)])[alphas[j]]).append(copy(fixpt))
                            ((regionsB[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            
                            A[fixpt] = alphas[j]
                            B[fixpt] = alphas[j+l]
                            seifert[fixpt]=1
                        
                            fixpt+= 1
                        
                            A[fixpt] = alphas[j+l]
                            B[fixpt] = alphas[j]
                            seifert[fixpt]=-1
        
                            (al[alphas[j+l]]).append(copy(fixpt))
                            (be[alphas[j]]).append(copy(fixpt))
    
                            ((regions[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            ((regionsB[(x0,x1)])[alphas[j]]).append(copy(fixpt))
                            
                            fixpt+= 1
                            #print(alphas[j])
                    else:
                        ep = -1
                        if abs(w[alphas[j]]) == abs(w[alphas[j+l]]):
                            ac[fixpt]= -ep
                            ac[fixpt+1]= -ep
                            ab[fixpt]= -ep
                            ab[fixpt+1]= -ep
                            (be[alphas[j]]).append(copy(fixpt))
                            (al[alphas[j+l]]).append(copy(fixpt))
    
                            ((regionsB[(x0,x1)])[alphas[j]]).append(copy(fixpt))
                            ((regions[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            
                            
                            B[fixpt] = alphas[j]
                            A[fixpt] = alphas[j+l]
                            seifert[fixpt]=1
                        
                            fixpt+= 1
                        
                            B[fixpt] = alphas[j+l]
                            A[fixpt] = alphas[j]
                            seifert[fixpt]=-1
        
                            (be[alphas[j+l]]).append(copy(fixpt))
                            (al[alphas[j]]).append(copy(fixpt))
    
                            ((regionsB[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            ((regions[(x0,x1)])[alphas[j]]).append(copy(fixpt))
        
                            
                            fixpt+= 1
                            #print(alphas[j])
                        else:
                            ac[fixpt]= ep
                            ac[fixpt+1]= ep
                            ab[fixpt]= ep
                            ab[fixpt+1]= ep
                            
                            (be[alphas[j]]).append(copy(fixpt))
                            (al[alphas[j+l]]).append(copy(fixpt))
    
                            ((regionsB[(x0,x1)])[alphas[j]]).append(copy(fixpt))
                            ((regions[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            
                            B[fixpt] = alphas[j]
                            A[fixpt] = alphas[j+l]
                            seifert[fixpt]=1
                        
                            fixpt+= 1
                        
                            B[fixpt] = alphas[j+l]
                            A[fixpt] = alphas[j]
                            seifert[fixpt]=-1
        
                            (be[alphas[j+l]]).append(copy(fixpt))
                            (al[alphas[j]]).append(copy(fixpt))
    
                            ((regionsB[(x0,x1)])[alphas[j+l]]).append(copy(fixpt))
                            ((regions[(x0,x1)])[alphas[j]]).append(copy(fixpt))
                            
                            fixpt+= 1
                            
                    
    
    ## The dictionaries regions and regionsB need to record the intersection points which appear along the various attaching points *in order*. 
    ## Currently, they are scrambled, since we followed the knot while labeling the intersection points. The last step is to reorder the points in regions and regionsB
    
    ordered_alphas = {x:[] for x in list_alpha_curves(w)}
    ordered_betas = {x:[] for x in list_alpha_curves(w)}
    
    for x in regions:
        #x records either the index of a handle [i] or the region between two handles (i, j) 
        if type(x) == int: ## i.e. if the intersection region is inside a handle
            ## there are two cases, depending on whether the crossing is positive or negative. 
            if w[x] > 0: 
                ## in this case, the alphas are easy -- each tuple of intersection points starts with an element of the fixed set, which may not be in the correct position. 
                ## All other intersection points have the correct relative order, so we just need to permute the fixed element in the ith strand (n-i) places to the right, where n is the number of strands.
                ## For the betas, we do the same
                regAx = regions[x]
                regBx = regionsB[x]
                for a in regAx:
                    ## a is an alpha curves in the xth handle
                    a_ints = ((regAx)[a])[1:]
                    b_ints = ((regBx)[a])[1:]                    
                    
                    a0 = ((regAx)[a])[0]
                    b0 = ((regBx)[a])[0]

                    n = len(regAx) -1 

                    ind_a = list(regAx.keys()).index(a)
                    ind_b = list(regBx.keys()).index(a)


                    new_alpha_ints = a_ints[0:n-ind_a]  + [a0] + a_ints[n-ind_a:]
                    new_beta_ints = b_ints[0:n-ind_b]  + [b0] + b_ints[n-ind_b:]

                    (ordered_alphas[a]).append({x:new_alpha_ints})
                    (ordered_betas[a]).append({x:new_beta_ints})
            else:
                ## if the crossing is negative, we need to first reverse the order of the intersection points. Then we move the fixed elements i elements to the right on the ith strand.
                regAx = regions[x]
                regBx = regionsB[x]
                for a in regAx:
                    ## a is an alpha curves in the xth handle
                    a_ints = ((regAx)[a])[1:]
                    b_ints = ((regBx)[a])[1:]   
                    a_ints.reverse()
                    b_ints.reverse()
                    
                    a0 = ((regAx)[a])[0]
                    b0 = ((regBx)[a])[0]

                    n = len(regAx) - 1

                    ind_a = list(regAx.keys()).index(a)
                    ind_b = list(regBx.keys()).index(a)


                    new_alpha_ints = a_ints[0:ind_a]  + [a0] + a_ints[ind_a:]
                    new_beta_ints = b_ints[0:ind_b]  + [b0] + b_ints[ind_b:]


                    (ordered_alphas[a]).append({x:new_alpha_ints})
                    (ordered_betas[a]).append({x:new_beta_ints})

        else:
            ## now we deal with the intersection points on the sth seifert disks between handles i and j. 
            ## Again, there are two cases, depending on the orientations of the individual curves. 
            ## For curves whose orientation disagrees with the orientation of the knot, we have to reverse the order of intersection points, and then permute the fixed point.
            x0 = x[0]
            x1 = x[1]

            ## In this case, we just permute the fixed points i places right in the ith strand
            regAx = regions[x]
            regBx = regionsB[x]
            for a in regAx:
                ## a is an alpha curves between handles x0 and x1
                a_ints = ((regAx)[a])[1:]
                b_ints = ((regBx)[a])[1:]   
                
                a0 = ((regAx)[a])[0]
                b0 = ((regBx)[a])[0]

                n = len(regAx) -1

                ind_a = list(regAx.keys()).index(a)
                ind_b = list(regBx.keys()).index(a)

                
                new_alpha_ints = a_ints[0:ind_a]  + [a0] + a_ints[ind_a:]
                new_beta_ints = b_ints[0:ind_b]  + [b0] + b_ints[ind_b:]

                if abs(w[a]) == max(abs(w[x1]), abs(w[x0])):
                    ## This is the case that the orientations of the alpha/beta curves disagree with the orientation of the knot. In these cases, we just reverse the order of the points.
                    new_alpha_ints.reverse()
                    new_beta_ints.reverse()

                (ordered_alphas[a]).append({x:new_alpha_ints})
                (ordered_betas[a]).append({x:new_beta_ints})
    
    new_ordered_alphas = {}
    new_ordered_betas = {}

    for x in ordered_alphas:
        new_ordered_alphas[x] = combine_dicts(ordered_alphas[x])
        new_ordered_betas[x] = combine_dicts(ordered_betas[x])

    # The next section of code takes an alpha (or beta) curve and returns an ordered list of intersection regions (handle intersection regions, or (i, j) Seifert intersection regions). 
    
    reg_path = {x:[] for x in list_alpha_curves(w)}
    for x in list_alpha_curves(w):
        ## First, we list the regions through which x passes -- this includes the two handles and the disk regions between them. We start at the handle labeled x. 
        ## x traverses two seirfert disks, which we call upper and lower
        upper = abs(w[x])-1 
        lower =abs(w[x])
        y = distinguished_pair(w,x)
        (reg_path[x]).append(x)
        upper_arcs = (enum_disk_arcs(w,upper)[0])
        upper_arcs.reverse()
        for r in upper_arcs:
            if r[-1] <= x:
                if r[0] not in distinguished_handles(w):
                    (reg_path[x]).append(tuple(r))
                else:
                    (reg_path[x]).append(tuple(r))
                    break
        (reg_path[x]).append(y)
        lower_arcs = (enum_disk_arcs(w,lower)[0])
        for r in lower_arcs:
            if r[0] >= y:
                if r[-1] != x:
                    (reg_path[x]).append(tuple(r))
                else:
                    (reg_path[x]).append(tuple(r))
                    break

    alpha_intersection_pts = {a:[] for a in list_alpha_curves(w)}
    beta_intersection_pts = {b:[] for b in list_alpha_curves(w)}

    for a in alpha_intersection_pts: ## a is an alpha curve
        path = reg_path[a] ## path is the ordered sequence of regions this curve paths
        for R in path: 
            if R in new_ordered_alphas[a]:
                Apts = (new_ordered_alphas[a])[R] ## this is the set of intersection points on a in the region R
                Bpts = (new_ordered_betas[a])[R]    
                alpha_intersection_pts[a] = alpha_intersection_pts[a]+Apts
                beta_intersection_pts[a] = beta_intersection_pts[a] + Bpts
    
    ints = {i:[A[i],B[i]] for i in range(fixpt)}

    if simplify == True:
        canceling_points = []
        ## There are four cases, depending on the sign of the crossing and whether the handle is distinguished or not.
        for handle in range(len(w)):
            crossing = w[handle]
            if handle in distinguished_handles(w):
                if sign(crossing) > 0:
                    ## Say w[i] = j. The next handle appearing on this disk corresponds to either [j] or [j-1]. 
                    ## In the second case, there are obvious cancelling intersection points -- every intersection point in the handle j will cancel with an intersection point appearing in the 
                    ## disk region [i, k], where k is the index of the next handle appearing in this disk.
                    for i in range(handle+1,len(w)):
                        v = w[i]
                        if abs(v) == abs(crossing):
                            break
                        elif abs(v) == abs(crossing) + 1:
                            arc = (handle, i) ## this is the arc between the distinguished handle and the next handle appearing on this disk region.
                            alphas_in_handle = handle_indices(w, crossing)[1:]
                            for alpha in alphas_in_handle:
                                #print(crossing, alphas_in_handle, alpha, new_ordered_alphas)
                                canceling_points = canceling_points + (new_ordered_alphas[alpha])[handle]
                                 # adds in the intersection points on the curve alpha in handle
                                ## Next, we remove the points on alpha which appear on arc
                                if arc in (new_ordered_alphas[alpha]):
                                    for beta in handle_indices(w, crossing)[1:]:
                                        canceling_points = canceling_points + [x for x in (new_ordered_alphas[alpha])[arc] if x in (new_ordered_betas[beta])[arc]]
                            break
                else:
                    for i in range(handle+1,len(w)):
                        v = w[i]
                        if abs(v) == abs(crossing):
                            break
                        elif abs(v) == abs(crossing) - 1:
                            arc = (handle, i) ## this is the arc between the distinguished handle and the next handle appearing on this disk region.
                            alphas_in_handle = handle_indices(w, crossing)[1:]
                            for alpha in alphas_in_handle:
                                canceling_points = canceling_points + (new_ordered_alphas[alpha])[handle]
                                #print((new_ordered_alphas[alpha])[handle])
                                ## Next, we remove the points on alpha which appear on arc
                                if arc in (new_ordered_alphas[alpha]):
                                    for beta in handle_indices(w, crossing)[1:]:
                                        canceling_points = canceling_points + [x for x in (new_ordered_alphas[alpha])[arc] if x in (new_ordered_betas[beta])[arc]]
                            break
            else:
                if sign(crossing) > 0:
                    previous_crossings = [j for j in range(handle)]
                    previous_crossings.reverse()
                    
                    for i in previous_crossings:
                        v = w[i]
                        if abs(v) == abs(crossing):
                            break
                        elif abs(v) == abs(crossing) - 1:
                            arc = (i, handle) ## this is the arc between the distinguished handle and the next handle appearing on this disk region.
                            
                            canceling_points = canceling_points + [x for x in (new_ordered_alphas[handle])[handle] if x in (new_ordered_betas[handle])[handle]] 
                            # adds in the intersection points on the curve alpha in handle 
                            # In the non-distinguished case, the index of the handle is the index of the only alpha curve through it.
                            
                            ## Next, we remove the points on alpha which appear on arc
                            if arc in (new_ordered_alphas[handle]):
                                canceling_points = canceling_points + [((new_ordered_alphas[handle])[arc])[0]]
                        break
                else:
                    previous_crossings = [j for j in range(handle)]
                    previous_crossings.reverse()
                    
                    for i in previous_crossings:
                        v = w[i]
                        if abs(v) == abs(crossing):
                            break
                        elif abs(v) == abs(crossing) + 1:
                            arc = (i, handle) ## this is the arc between the distinguished handle and the next handle appearing on this disk region.
                            canceling_points = canceling_points + (new_ordered_alphas[handle])[handle]
                            #print((new_ordered_alphas[handle])[handle])
                            # adds in the intersection points on the curve alpha in handle 
                            # In the non-distinguished case, the index of the handle is the index of the only alpha curve through it.
                            
                            ## Next, we remove the points on alpha which appear on arc
                            if arc in (new_ordered_alphas[handle]):
                                canceling_points = canceling_points + [((new_ordered_alphas[handle])[arc])[-1]]
                        break
                            
        #print(canceling_points)
        if len(canceling_points) - len(set(canceling_points)) != 0:
            print("Cancellation error!")
        return [filter_dict_values(al, canceling_points),
                filter_dict_values(be, canceling_points),
                filter_dict_values(alpha_intersection_pts, canceling_points),
                filter_dict_values(beta_intersection_pts, canceling_points),
                remove_keys_from_dict(tau, canceling_points),
                remove_keys_from_dict(ab, canceling_points),
                remove_keys_from_dict(ac, canceling_points),
                remove_keys_from_dict(ints, canceling_points),
                N,
                remove_keys_from_dict(seifert, canceling_points),
               ]
                
    elif test == True:
        return count_int_pts(w) == len(A) and count_fix_pts(w) == N
    else:
        #return alpha_intersection_pts, beta_intersection_pts
        return al,be, alpha_intersection_pts, beta_intersection_pts, tau, ab, ac, ints, N, seifert
        ## Too many outputs, but here's the lowdown:

        ## 0, 1: al (be) inputs an alpha (beta) curve a, and outputs intersection points on a (the order follows the path of the knot)
        ## 2, 3: alpha_intersection_pts (beta_intersection_pts) inputs an alpha (beta) curves and outputs the intersection points in order the appear on the curve,
        ## 4: tau is the involution
        ## 5: ab is the intersection number a*b
        ## 6: ac is the intersection number of a with the fixed set
        ## 7: ints inputs an intersection number and outputs [i, j] encoding the alpha and beta curve it lies on
        ## 8: N is the number of fixed elements
        
        
def real_HD(w):
    """
    returns a Real_heegaard_diagram object associated to a knot K given by the braid closure of w. 
    """
    H = make_HD(w)
    return Real_Heegaard_Diagram(H[2],H[3],H[4],H[5],H[6])



In [235]:
# 4_1 is the braid closure of the braid [1,-2,1,-2]

H = real_HD([1,-2,1,-2])
#print(H.alphas, H.betas,H.tau, H.alpha_int_beta, H.alpha_int_C)
#H.homology()
#H.real_euler_characteristics()
#H.real_euler_characteristics()

In [234]:
# Braid words for some small crossing knots.

#'3_1': [-1, -1, -1],
#'4_1': [1, -2, 1, -2],
#'5_1': [-1, -1, -1, -1, -1],
#'5_2': [-1, 2, -3, 2, 2, 1, 2, 3, 2],
#'6_2': [1, -2, 1, 1, 1, -2],
#'6_3': [1, -2, 1, -2, -2, 1],
#'7_1': [-1, -1, -1, -1, -1, -1, -1],
#'7_7': [1, -2, 3, -2, 1, -2, 1, -3, -2]
